# CHATGPT api 활용해서 수행한 과제

#### 연결 테스트

In [ ]:
! pip install openai

In [6]:
from openai import OpenAI

In [7]:
# 개인 키 발급
OPENAI_API_KEY = ""

In [8]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [9]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "안녕"}
    ], temperature = 0.8
)

In [11]:
results = completion.choices[0].message.content
print(results)

안녕하세요! 무엇을 도와드릴까요?


In [ ]:
# 적용 가능한 파라미터   
# temperature=0.2 -- 0~2.0 사이 (2.0 일수록 다양한 답변 생성)
# top_p=0.5 -- 0~1 사이 (0일수록 일반적으로 예측할 수 있는 답변 생성)
# max_tokens=1000 # 최대 토근 수 지정 gpt-3.5-turbo = 4096 max

#### 과제
데이터: 독후감 (초등학생 고학년 작성) -> 내 3가지 독후감 샘플 제공 (외부 데이터 사용도 상관없음)  
목적: 독후감 첨삭 프로그램 개발  
방식: 독후감 첨삭을 위한 gpt api 프롬프트 엔지니어링  


평가1 (필수):  
1. '문장' 단위로 접근 (chat gpt 단독 사용이 아닌, 외부 시스템 개발을 위해서임)
2. 독후감 내 글쓴이가 '느낀점'(감정) 이 있는 문장을 찾기 (줄거리에 대한 내용은 제외)  
3. 느낀점이 있는 문장에 대해서는 감정 '키워드'를 찾아야함 (다중 키워드도 가능)
4. [독후감]  
"주인공 은석이는 학교를 다닌다. 나는 은석이가 배가 고파서 굶을때마다 너무 마음이 아팠다. 은석이가 꼭 행복해지길 빈다."  
문장1: "주인공 은석이는 학교를 다닌다." -> "없음"  
문장2: "나는 은석이가 배가 고파서 굶을때마다 너무 마음이 아팠다." -> "슬픔"   
문장3: "은석이가 꼭 행복해지길 빈다." -> "행복, 소망"
5. 문장 별로 개선 사항을 제시함
6. 결과에 포함될 내용: 문장 index, 문장, 해당 문장에 존재하는 감정 키워드, 개선 내용 제안 -> 포맷은 자유  

평가2 (옵션) :  
추가 아이디어 제시 영역  
독후감 첨삭을 위해 추가적으로 필요한 부분을 자유롭게 분석 -> 예) '총평', '요약', '문법'  

** 자연어 처리 모듈과 조합하여 활용해도 무방함  
** gemini(Google) 등 무료 API 활용도 가능

---
# 프롬프트 정의
---

In [1]:
# 프롬프트 초기화
system_prompt = ""
user_prompt = ""

In [45]:
# 로컬 저장소에 있는 텍스트 파일 경로
file_paths = [
    r"./미래가온다,뇌과학.txt",
    r"./와이파이를먹어버린펄럭거리는바지들.txt",   
    r"./책과노니는집.txt",   
]

# 선택할 파일의 인덱스
selected_file_index = 0  # 전체 파일을 읽어오면 내용이 짤려 한 파일씩 선택하게 지정

# 선택한 파일 경로
file_path = file_paths[selected_file_index]

# 파일 경로에서 파일을 읽어와서 user_prompt에 추가

# 파일 읽기
with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()

# 파일 이름에서 책 제목 추출
book_title = file_path.split("\\")[-1][:-4]

# 파일 별로 구분자를 추가하여 user_prompt에 저장
user_prompt += f"<{selected_file_index + 1}. 책 제목: {book_title}>\n{content}\n\n"

In [ ]:
# 내용 확인
user_prompt

In [104]:
# system_prompt 정의
system_prompt = """당신은 초등학교 고학년의 글에서 감정표현을 파악하고, 정확한 문장을 쓰게 하기 위해 독후감 첨삭 프로그램을 개발하는 개발자입니다.
이 프로그램의 목적은 감정 표현을 파악하고 맞춤법을 검사하여 최적의 문장을 제공하는 서비스를 개발하기 위한 것입니다.
첨삭 내용 이외에는 최대한 작성된 내용을 유지합니다.

이 독후감 감정 첨삭 프로그램 개발은 몇 가지의 규칙이 있습니다.
규칙 1. 처리 결과는 다음과 같이 나타냅니다.
<책 제목>

문장 번호 | 원 문장 | 감정 키워드 | 맞춤법 검사 | 추천 문장
|---|---|---|---|---|
|번호|내용|내용|내용|내용|
|---|---|---|---|---|

내용이 없는 경우는 '없음'으로 작성해 표에 밀리는 현상이 없도록 해야 합니다.
'원 문장'은 아무 변형도 가하지 않은 문장을 그대로 작성합니다.
'첨삭 문장'에는 수정한 문장을 작성합니다.
생략하지 않고 모든 내용을 보이게 합니다.

규칙 2. 문장 단위로 검토해야 합니다.
    세부규칙 1. 개행 문자 \n를 무시합니다.
    세부규칙 2. 처음부터 가장 처음 나오는 온점('.')이나 느낌표, 물음표 등 마침표('!', '?') 까지를 한 문장으로 보고 끊어가며 첨삭합니다.
    세부규칙 3. 세부규칙 1과 세부규칙2에 해당하지 않을 경우 맺음말(했다, 였다, 이었다, 좋겠다, 있었다 등)로 끊어가며 첨삭합니다.

규칙 3. 문장을 파악하여 '느낀점'(감정)이 있는 키워드를 찾아 '감정 키워드'에 기록합니다.
    세부규칙 1. 감정은 '슬픔', '분노', '화남' 등 명확하게 제시해야 합니다.
    세부규칙 2. 감정이 없을 경우 '감정 키워드'에 '없음' 으로 기록합니다.
    세부규칙 3. 모든 문장을 검토해 처리 결과에 나와야 합니다.

규칙 4. 책 내용 안에 있는 감정은 '없음' 으로 표시합니다.
    예시문장 : "형준이는 PC가 망가져 슬퍼했다" -> 없음
              "형준이의 PC가 망가지는 장면을 읽고 슬펐다" -> 슬픔
              "왜 형준이의 친구는 그렇게 해야만 했을까? 화가 난다" -> 분노

규칙 5. 문장 별로 맞춤법을 검사해 수정 내용을 제시해야 합니다.
    세부규칙 1. 맞춤법, 주술관계, 오타 등을 확인해 '맞춤법 검사'에 기록합니다. 
    세부규칙 2. '맞춤법 검사'는 맞춤법이나 문장의 적격 여부, 오타를 판단하고 문장이 아닌 예시와 같이 작성합니다.
                -> 예시 : 혜야겠다 -> 해야겠다, 슬플겠다 -> 슬프겠다, 서술어와 주어가 불일치, 오타

규칙 6. 첨삭이 끝나면 전체적인 개선사항과 독후감 내에서 부족했거나 추천하는 감정 표현을 세 줄 내로 작성합니다.
        -> 예시 : - 전체적인 개선사항: 맞춤법 및 띄어쓰기에 주의하며 문장 구조를 좀 더 명확하게 표현하면 독후감이 더욱 독자 친화적으로 전달될 것입니다.
               - 추천 감정표현: 호기심, 열정, 궁금함, 흥미, 감탄
"""

---
# 
system prompt에 전처리 과정을 설명했으나, 외부 이용 시 문제가 있을 수 있으므로 전처리 수행 코드도 따로 작성했습니다.

---

In [103]:
def preprocess_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # 문장 구분 기준으로 문장을 나누기
    sentences = []
    current_sentence = ""
    for i, char in enumerate(text):
        # 마침표, 느낌표, 물음표가 나오면 문장 종료
        if char in ['.', '!', '?']:
            current_sentence += char 
            sentences.append(current_sentence.strip())
            current_sentence = ""
        # '다'가 나오면 문장 종료
        # elif char == '다':
        #     # '다'가 마지막 문자가 아니고 '다면', '다고', '다가'가 나오면 문장으로 분리 안함
            if i < len(text) - 1 and text[i+1:i+3] in ['면', '고', '가']:
                current_sentence += char
        #     # '다' 뒤에 '하다'가 아니고 '사람마다'가 아니면 문장으로 처리
            elif i == len(text) - 1 or (text[i+1:i+4] != '하다' and text[i+1:i+5] != '사람마다'):
        #         # '다' 뒤에 아무것도 없거나 '사람마다'가 나오면 문장으로 분리 안함
                if i == len(text) - 1 or text[i+1] in ['"', "'", ")", "]", "}", ">", " ", "\n"] or text[i+1:i+4] == '다 ':
                    current_sentence += ""
        # 개행 문자는 무시
        elif char == '\n':
            continue
        # 그 외의 경우 문장에 추가
        else:
            current_sentence += char

    # 마지막 문장 추가
    if current_sentence:
        sentences.append(current_sentence.strip())
    
    return sentences

# 선택할 파일의 인덱스
selected_file_index = 0  # 전체 파일을 읽어오면 내용이 짤려 한 파일씩 선택하게 지정

# 선택한 파일 경로
file_path = file_paths[selected_file_index]
processed_sentences = preprocess_text(file_path)

# 문장 출력
for idx, sent in enumerate(processed_sentences):
    print(f"Sentence {idx + 1}: {sent}")


Sentence 1: <미래가 온다, 뇌 과학> 이 책은 뇌에 대해 알 수있는 책이다.
Sentence 2: 짧고 간략한 내용이지만 톡톡 튀는 재미도 느끼며 수많은 뇌의 정보와 뇌의 역할,뇌가 어떻게 이루어져 있는지 다 알수있다!
Sentence 3: 독자들은 이 책을 읽으면서 뇌의 신비함과 여러가지 과학잡식들을 배우 면서 느낄 수있다.
Sentence 4: 나는 이 책을 읽으면서뉴런과 피걸에 대해 알 수있었다.
Sentence 5: 어떤가,흥미롭지 않은가?
Sentence 6: 뉴런은 뇌에 들어있는 세포들이다.
Sentence 7: 뉴런은 연합뉴런,신경뉴런,운동뉴런이 있다.
Sentence 8: 7개의 뉴런은 1000개의 또다른 뉴런들과 연결될수 있다.
Sentence 9: 이 뉴런들이 서로 연결 될 수있는 신경고리가 있는데 이 신경고리를 커넥텀이라 한다사람마다 서로 생각,느낌,감정들이 다른 이유는이 커넥톰이 사람마다 무조건 각각 다르기 때문이다.
Sentence 10: 또한 사람들이 서로의 경험이 다르고 커넥톰이 다른 것도 한몫한다.
Sentence 11: 동물들에 따라 뇌에 있는 피질이 있는데 이 피질이 동물의 기억력을좌지우지 한다.
Sentence 12: 개구리,물고기,악어의 피질은 거의 뇌가 매끈매끈하기 때문에 거의 없다.
Sentence 13: 그래서그들은 장기기억력이 없어 지금밖에 기억을 할 수 없다.
Sentence 14: 하지만 쥐는 조금 더 뇌가 크기 때문에 전에 있었던 일들을 기억할 수 있다.
Sentence 15: 하지만사람에 비하면 아무것도 아니다.
Sentence 16: 사람은 뇌가 쭈굴쭈굴 주름졌다.
Sentence 17: 따라서 우리는 장기기억력이 상당히 다른동물에 비하면 높고 미래를 생각하고 행동하며 상상할 수 있다.
Sentence 18: 이렇게 많은 내용을알 수 있으니 나는 이책을 정말 재밌게 읽었다.


---
# GPT-3.5-turbo
---

In [105]:
# 결과 출력
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}, 
    ], temperature = 0.5, top_p= 0.5 # 파라미터 조정
)
results = completion.choices[0].message.content
print(results)

<미래가 온다, 뇌 과학>

| 문장 번호 | 원 문장 | 감정 키워드 | 맞춤법 검사 | 추천 문장 |
|---|---|---|---|---|
|1|이 책은 뇌에 대해 알 수있는 책이다.|없음|없음|없음|
|2|짧고 간략한 내용이지만 톡톡 튀는 재미도 느끼며 수 많은 뇌의 정보와 뇌의 역할,뇌가 어떻게 이루 어져 있는지 다 알수있다!|흥미|수 많은 뇌의 정보와 뇌의 역할, 뇌가 어떻게 이루어져 있는지 다 알 수 있다!|
|3|독자들은 이 책을 읽으면서 뇌의 신비함과 여러가지 과학잡식들을 배 우 면서 느낄 수있다.|흥미|과학잡식|독자들은 이 책을 읽으면서 뇌의 신비함과 여러 가지 과학잡식들을 배우며 느낄 수 있다.|
|4|나는 이 책을 읽으면서 뉴런과 피걸에 대해 알 수있었다.|없음|피걸|나는 이 책을 읽으면서 뉴런과 피걸에 대해 알 수 있었다.|
|5|어떤가,흥미롭 지 않은가?|흥미|어떤가, 흥미로운가?|어떤가, 흥미로운가?|
|6|뉴런은 뇌에 들어있는 세포들이다.|없음|없음|없음|
|7|뉴런은 연합뉴런,신경뉴런,운동뉴런이 있다.|없음|없음|없음|
|8|7개의 뉴런은 1000개의 또다른 뉴런들과 연결될 수 있다.|없음|없음|없음|
|9|이 뉴런들이 서로 연결 될 수있는 신 경고리가 있는데 이 신경고리를 커넥텀이라 한다|없음|신경고리|이 뉴런들이 서로 연결될 수 있는 신경고리가 있는데, 이를 커넥텀이라 한다.|
|10|사람마다 서로 생각,느낌,감정들이 다른 이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문 이다.|없음|이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문이다.|사람마다 서로 생각, 느낌, 감정들이 다른 이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문이다.|
|11|또한 사람들이 서로의 경험이 다르고 커넥 톰이 다른 것도 한몫한다.|없음|한몫한다|또한 사람들이 서로의 경험이 다르고 커넥톰이 다른 것도 한몫한다.|
|12|동물들에 따라 뇌에 있는 피질이 있는데 이 피질이 동물의 기억력을 좌지우지 한다.|없음|없음|없음|
|13|개구리,물고

<미래가 온다, 뇌 과학>

| 문장 번호 | 원 문장 | 감정 키워드 | 맞춤법 검사 | 추천 문장 |
|---|---|---|---|---|
|1|이 책은 뇌에 대해 알 수있는 책이다.|없음|없음|없음|
|2|짧고 간략한 내용이지만 톡톡 튀는 재미도 느끼며 수 많은 뇌의 정보와 뇌의 역할,뇌가 어떻게 이루 어져 있는지 다 알수있다!|흥미|수 많은 뇌의 정보와 뇌의 역할, 뇌가 어떻게 이루어져 있는지 다 알 수 있다!|
|3|독자들은 이 책을 읽으면서 뇌의 신비함과 여러가지 과학잡식들을 배 우 면서 느낄 수있다.|흥미|과학잡식|독자들은 이 책을 읽으면서 뇌의 신비함과 여러 가지 과학잡식들을 배우며 느낄 수 있다.|
|4|나는 이 책을 읽으면서 뉴런과 피걸에 대해 알 수있었다.|없음|피걸|나는 이 책을 읽으면서 뉴런과 피걸에 대해 알 수 있었다.|
|5|어떤가,흥미롭 지 않은가?|흥미|어떤가, 흥미로운가?|어떤가, 흥미로운가?|
|6|뉴런은 뇌에 들어있는 세포들이다.|없음|없음|없음|
|7|뉴런은 연합뉴런,신경뉴런,운동뉴런이 있다.|없음|없음|없음|
|8|7개의 뉴런은 1000개의 또다른 뉴런들과 연결될 수 있다.|없음|없음|없음|
|9|이 뉴런들이 서로 연결 될 수있는 신 경고리가 있는데 이 신경고리를 커넥텀이라 한다|없음|신경고리|이 뉴런들이 서로 연결될 수 있는 신경고리가 있는데, 이를 커넥텀이라 한다.|
|10|사람마다 서로 생각,느낌,감정들이 다른 이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문 이다.|없음|이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문이다.|사람마다 서로 생각, 느낌, 감정들이 다른 이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문이다.|
|11|또한 사람들이 서로의 경험이 다르고 커넥 톰이 다른 것도 한몫한다.|없음|한몫한다|또한 사람들이 서로의 경험이 다르고 커넥톰이 다른 것도 한몫한다.|
|12|동물들에 따라 뇌에 있는 피질이 있는데 이 피질이 동물의 기억력을 좌지우지 한다.|없음|없음|없음|
|13|개구리,물고기,악어의 피질은 거의 뇌가 매끈매끈하기 때문에 거의 없다.|없음|없음|없음|
|14|그래서 그들은 장기기억력이 없어 지금밖에 기억을 할 수 없다.|없음|없음|없음|
|15|하지만 쥐는 조금 더 뇌가 크기 때문에 전에 있었던 일들을 기억할 수 있다.|없음|없음|없음|
|16|하지만 사람에 비하면 아무것도 아니다.|없음|없음|없음|
|17|사람은 뇌가 쭈 굴쭈굴 주름졌다.|없음|굴쭈굴|사람은 뇌가 굴쭈굴 주름졌다.|
|18|따라서 우리는 장기기억력이 상 당히 다른동물에 비하면 높고 미래를 생각하고 행동하며 상상할 수 있다.|없음|상당히|따라서 우리는 장기기억력이 상당히 다른 동물에 비하면 높고, 미래를 생각하고 행동하며 상상할 수 있다.|
|19|이렇게 많은 내용을 알 수 있으니 나는 이책을 정말 재밌게 읽었다.|흥미|이렇게 많은 내용을 알 수 있으니 나는 이 책을 정말 재밌게 읽었다.|이렇게 많은 내용을 알 수 있어서 나는 이 책을 정말 재밌게 읽었다.|

- 전체적인 개선사항: 문장 구조를 좀 더 명확하게 표현하고 맞춤법 및 띄어쓰기에 주의하여 작성하면 독후감이 더욱 독자 친화적으로 전달될 것입니다.
- 추천 감정 표현: 흥미, 신기함, 감탄

---
# 파라미터 temperature 0.2, top_p 0.1 일 때   
<미래가 온다, 뇌 과학>

| 문장 번호 | 원 문장 | 감정 키워드 | 맞춤법 검사 | 추천 문장 |
|---|---|---|---|---|
|1|이 책은 뇌에 대해 알 수있는 책이다.|없음|없음|없음|
|2|짧고 간략한 내용이지만 톡톡 튀는 재미도 느끼며 수 많은 뇌의 정보와 뇌의 역할,뇌가 어떻게 이루 어져 있는지 다 알수있다!|흥미|수 많은 -> 수많은|짧고 간략한 내용이지만 톡톡 튀는 재미도 느끼며 수많은 뇌의 정보와 뇌의 역할, 뇌가 어떻게 이루어져 있는지 다 알 수 있다!|
|3|독자들은 이 책을 읽으면서 뇌의 신비함과 여러가지 과학잡식들을 배 우 면서 느낄 수있다.|흥미|배 우 -> 배우|독자들은 이 책을 읽으면서 뇌의 신비함과 여러가지 과학잡식들을 배우면서 느낄 수 있다.|
|4|나는 이 책을 읽으면서 뉴런과 피걸에 대해 알 수있었다.|흥미|피걸 -> 피겨|나는 이 책을 읽으면서 뉴런과 피겨에 대해 알 수 있었다.|
|5|어떤가,흥미롭 지 않은가?|흥미|흥미롭 -> 흥미로운|어떤가, 흥미로 지 않은가?|
|6|뉴런은 뇌에 들어있는 세포들이다.|없음|들어있는 -> 들어 있는|뉴런은 뇌에 들어 있는 세포들이다.|
|7|뉴런은 연합뉴런,신경뉴런,운동뉴런이 있다.|없음|없음|없음|
|8|7개의 뉴런은 1000개의 또다른 뉴런들과 연결될 수 있다.|없음|없음|없음|
|9|이 뉴런들이 서로 연결 될 수있는 신 경고리가 있는데 이 신경고리를 커넥텀이라 한다|없음|신 경고리 -> 신경 고리|이 뉴런들이 서로 연결될 수 있는 신경 고리가 있는데 이 신경고리를 커넥텀이라 한다.|
|10|사람마다 서로 생각,느낌,감정들이 다른 이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문 이다.|없음|이유는 -> 이유는|사람마다 서로 생각, 느낌, 감정들이 다른 이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문이다.|
|11|또한 사람들이 서로의 경험이 다르고 커넥 톰이 다른 것도 한몫한다.|없음|커넥 톰 -> 커넥톰|또한 사람들이 서로의 경험이 다르고 커넥톰이 다른 것도 한몫한다.|
|12|동물들에 따라 뇌에 있는 피질이 있는데 이 피질이 동물의 기억력을 좌지우지 한다.|없음|없음|없음|
|13|개구리,물고기,악어의 피질은 거의 뇌가 매끈매끈하기 때문에 거의 없다.|없음|없음|없음|
|14|그래서 그들은 장기기억력이 없어 지금밖에 기억을 할 수 없다.|없음|지금밖에 -> 지금 밖에|그래서 그들은 장기기억력이 없어 지금 밖에 기억을 할 수 없다.|
|15|하지만 쥐는 조금 더 뇌가 크기 때문에 전에 있었던 일들을 기억할 수 있다.|없음|없음|없음|
|16|하지만 사람에 비하면 아무것도 아니다.|없음|없음|없음|
|17|사람은 뇌가 쭈 굴쭈굴 주름졌다.|없음|쭈 굴쭈굴 -> 쭈굴쭈굴|사람은 뇌가 쭈굴쭈굴 주름졌다.|
|18|따라서 우리는 장기기억력이 상 당히 다른동물에 비하면 높고 미래를 생각하고 행동하며 상상할 수 있다.|없음|상 당히 -> 상당히|따라서 우리는 장기기억력이 상당히 다른 동물에 비하면 높고 미래를 생각하고 행동하며 상상할 수 있다.|
|19|이렇게 많은 내용을 알 수 있으니 나는 이책을 정말 재밌게 읽었다.|흥미|없음|없음|

- 전체적인 개선사항: 문장 구조를 좀 더 간결하고 명확하게 표현하면 독후감이 더욱 효과적으로 전달될 것입니다.
- 추천 감정표현: 호기심, 놀라움, 감탄, 흥미, 사실감

---
---
# 총평
- 파라미터를 강하게 조정해서인지 답하는 감정의 종류가 매우 적었음  
- 파라미터를 주지 않거나 다양한 대답을 하게 조정한다면 더 다양한 답을 이끌어 낼 수 있었음
- 돌릴수록 정확도가 높아지지 않고 답이 중구난방으로 나옴. system prompt를 조정하면 더 심해짐
---
---

---
# GPT-4
---

In [ ]:
# 결과 출력
completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)
results = completion.choices[0].message.content
print(results)

<미래가 온다, 뇌 과학>

문장 번호 | 원 문장 | 감정 키워드 | 맞춤법 검사 | 추천 문장
|---|---|---|---|---|
|1|이 책은 뇌에 대해 알 수 있는 책이다|호기심|'수있는' -> '수 있는' 중간 공백 추가필요|'이 책은 뇌에 대해 알 수 있는 책이다'|
|2|짧고 간략한 내용이지만 톡톡 튀는 재미도 느낌|재미|'느낌' -> '느낄 수 있음' 및 조사 필요|'짧고 간략한 내용이지만 톡톡 튀는 재미도 느낄 수 있었다'|
|3|이 책을 읽으면서 뇌의 신비함과 여러가지 과학잡식들을 배우 면서 느낌|신비함|'과학잡식들' -> '과학적 사실들' 수정 필요|'이 책을 읽으면서 뇌의 신비함과 여러 가지 과학적 사실들을 배울 수 있었다'|
|4|뉴런과 피걸에 대해 알 수 있었다|흥미|'피걸' -> '피질' 오타|'뉴런과 피질에 대해 알 수 있었다'|
|5|뉴런은 연합뉴런,신경뉴런,운동뉴런이 있다|감탄|'연합뉴런,신경뉴런,운동뉴런' -> '연합 뉴런, 신경 뉴런, 운동 뉴런' 공백 추가 필요|'뉴런은 연합 뉴런, 신경 뉴런, 운동 뉴런이 있다'|
|6|7개의 뉴런은 1000개의 또 다른 뉴런들과 연결될 수 있다|놀라움|없음|'7개의 뉴런은 1000개의 또 다른 뉴런들과 연결될 수 있다'|
|7|사람마다 서로 생각, 느낌, 감정들이 다른 이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문이다 |호기심|없음|'사람마다 서로 생각, 느낌, 감정들이 다른 이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문이다'|
|8|개구리, 물고기, 악어의 피질은 거의 뇌가 매끈매끈하기 때문에 거의 없다|놀라움|없음|'개구리, 물고기, 악어의 피질은 거의 뇌가 매끈매끈하기 때문에 거의 없다'|
|9|사람의 뇌는 주름져 있어 서 우리는 장기기억력이 상당히 다른 동물에 비하면 높고|감탄|'주름져 있어 서' -> '주름져 있어서' 띄어쓰기 오류, 감정 표현 부족|'사람의 뇌는 주름져 있어서 우리는 장기기억력이 상당히 높다는 것을 이해할 수 있었다'|
|10|이렇게 많은 내용을 알 수 있으니 나는 이 책을 정말 재밌게 읽었다|재미|없음|'이렇게 많은 내용을 알 수 있으니 나는 이 책을 정말 재밌게 읽었다'|

- 전체적인 개선사항: '흥미', '감탄', '재미', '놀라움' 같은 감정 표현을 적극적으로 활용하면 독자에게 책에 대한 흥미를 더욱 끌 수 있을 것 같습니다.
- 부족한 감정표현: '믿음', '희망', '존경', '기대', '황홀함' 등 이 책을 통해 얻을 수 있는 긍정적인 감정을 더욱 표현해보세요.

---
---
# 총평
- 3.5 turbo에 비해 깔끔하고 간결한 문장
- 감정표현이 없는 문장임에도 없음이 아닌 어떻게든 답을 찾으려고 하는 것처럼 보였음
- 문장을 모두 나오게 해달라고 규칙을 세웠음에도 임의로 문장을 선택함
---
---

---
# google-generativeai
---

In [ ]:
pip install google-cloud-ai-generation

In [23]:
import os
import google.generativeai as genai
api_key = ""  # 여기에 자신의 API 키를 넣어주세요

genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-pro')


In [155]:
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
)

In [ ]:
response = model.generate_content(system_prompt + user_prompt)
print(response.text)

<미래가 온다, 뇌 과학>

| 문장 번호 | 원 문장 | 감정 키워드 | 맞춤법 검사 | 추천 문장 |
|---|---|---|---|---|
| 1 | 이 책은 뇌에 대해 알 수있는 책이다. | 없음 | 없음 | 이 책은 뇌를 알 수 있는 책이다. |
| 2 | 짧고 간략한 내용이지만 톡톡 튀는 재미도 느끼며 | 없음 | 없음 | 짧고 간략하지만 재미도 느낄 수 있다. |
| 3 | 수많은 뇌의 정보와 뇌의 역할,뇌가 어떻게 이루어져 있는지 다 알수있다! | 없음 | '뇌가' 중복 사용 | 수많은 뇌 정보와 역할, 구성을 알 수 있다. |
| 4 | 독자들은 이 책을 읽으면서 뇌의 신비함과 여러가지 과학잡식들을 배워 면서 느낄 수있다. | 없음 | '배워 면서' -> '배우면서' 주격과 목적격 불일치 | 독자들은 이 책을 읽으면서 뇌의 신비로움과 과학 정보를 배울 수 있다. |
| 5 | 나는 이 책을 읽으면서 뉴런과 피걸에 대해 알 수있었다. | 없음 | '피걸' -> '피질' | 나는 이 책을 읽으면서 뉴런과 피질에 대해 알게 되었다. |
| 6 | 어떤가,흥미롭지 않은가? | 없음 | '흥미롭지 않은가?' -> '흥미롭지 않은가.' | 어떤가, 흥미롭지 않은가? |
| 7 | 뉴런은 뇌에 들어있는 세포들이다. | 없음 | 없음 | 뉴런은 뇌에 있는 세포다. |
| 8 | 뉴런은 연합뉴런,신경뉴런,운동뉴런이 있다. | 없음 | 없음 | 뉴런은 연합뉴런, 신경뉴런, 운동뉴런으로 나뉜다. |
| 9 | 7개의 뉴런은 1000개의 또다른 뉴런들과 연결될 수 있다. | 없음 | 없음 | 7개의 뉴런은 1000개의 다른 뉴런과 연결될 수 있다. |
| 10 | 이 뉴런들이 서로 연결 될 수있는 신경고리가 있는데 이 신경고리를 커넥텀이라 한다 | 없음 | 없음 | 이 뉴런들이 서로 연결되는 신경 고리가 커넥텀이다. |
| 11 | 사람마다 서로 생각,느낌,감정들이 다른 이유는 이 커넥톰이 사람마다 무조건 각각 다르기 때문이다. | 없음 | 없음 | 사람마다 생각, 느낌, 감정이 다른 이유는 커넥텀이 각기 다르기 때문이다. |
| 12 | 또한 사람들이 서로의 경험이 다르고 커넥톰이 다른 것도 한몫한다. | 없음 | 없음 | 사람마다 다른 경험과 커넥텀도 영향을 미친다. |
| 13 | 동물들에 따라 뇌에 있는 피질이 있는데 이 피질이 동물의 기억력을 좌지우지 한다. | 없음 | 없음 | 동물의 뇌에는 피질이 있어서 기억력에 영향을 미친다. |
| 14 | 개구리,물고기,악어의 피질은 거의 뇌가 매끈매끈하기 때문에 거의 없다. | 없음 | 없음 | 개구리, 물고기, 악어는 뇌가 매끈매끈해서 피질이 거의 없다. |
| 15 | 그래서 그들은 장기기억력이 없어 지금밖에 기억을 할 수 없다. | 없음 | 없음 | 그래서 장기 기억력이 없고 현재만 기억할 수 있다. |
| 16 | 하지만 쥐는 조금 더 뇌가 크기 때문에 전에 있었던 일들을 기억할 수 있다. | 없음 | 없음 | 하지만 쥐는 뇌가 조금 더 크기 때문에 과거 일들을 기억할 수 있다. |
| 17 | 하지만 사람에 비하면 아무것도 아니다. | 없음 | 없음 | 하지만 사람에 비하면 기억력이 훨씬 떨어진다. |
| 18 | 사람은 뇌가 쭈굴쭈굴 주름졌다. | 없음 | 없음 | 사람의 뇌는 주름이 많이 졌다. |
| 19 | 따라서 우리는 장기기억력이 상당히 다른동물에 비하면 높고 미래를 생각하고 행동하며 상상할 수 있다. | 없음 | 없음 | 따라서 우리는 다른 동물에 비해 장기 기억력이 뛰어나고 미래를 생각하고 행동하며 상상할 수 있다. |
| 20 | 이렇게 많은 내용을 알 수 있으니 나는 이책을 정말 재밌게 읽었다. | 없음 | 없음 | 이렇게 많은 내용을 알 수 있어서 나는 이 책을 정말 재미있게 읽었다. |

- 전체적인 개선사항: 맞춤법 및 띄어쓰기에 주의하며 문장 구조를 좀 더 명확하게 표현하면 독후감이 더욱 독자 친화적으로 전달될 것입니다.
- 추천 감정표현: 호기심, 열정, 궁금함, 흥미, 감탄

---
---
# 총평
- 사용법이 간단함
- 규칙 그대로 옮겨 적는 경우가 많았음
- 파라미터를 조절할 수 있는 방법을 찾지 못했음.
---
---

---
# 독후감 영어 번역 + 단어장 단들기 프로그램 개발
---

- 본인이 작성한 글이 영어로 번역되고, 그중 학력 수준에 맞는 영단어를 추출해 영어에 친숙하게 하는 프로그램 입니다.

In [27]:
user_prompt_e = ""

In [28]:
# 로컬 저장소에 있는 텍스트 파일 경로
file_paths = [
    r"./미래가온다,뇌과학.txt",
    r"./와이파이를먹어버린펄럭거리는바지들.txt",   
    r"./책과노니는집.txt",   
]

# 선택할 파일의 인덱스
selected_file_index = 1  # 전체 파일을 읽어오면 내용이 짤려 한 파일씩 선택하게 지정

# 선택한 파일 경로
file_path = file_paths[selected_file_index]

# 파일 경로에서 파일을 읽어와서 user_prompt에 추가

# 파일 읽기
with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()

# 파일 이름에서 책 제목을 추출합니다.
book_title = file_path.split("\\")[-1][:-4]
# 파일 별로 구분자를 추가하여 user_prompt에 저장합니다.
user_prompt_e += f"<{selected_file_index + 1}. 책 제목: {book_title}>\n{content}\n\n"

In [25]:
# system_prompt_e 정의
system_prompt_e = """당신은 초등학교 고학년의 글에서 자신이 쓴 글을 영어로 번역하고, 초등학생~ 중학생이 알아야 하는 필수 단어를 출력하는 독후감 프로그램을 개발하는 개발자입니다.
이 프로그램의 목적은 영어 번역과 단어장을 제공하는 서비스를 개발하기 위한 것입니다.

이 독후감 감정 첨삭 프로그램 개발은 몇 가지의 규칙이 있습니다.
규칙 1. 처리 결과는 다음과 같이 나타냅니다.
요약 문장 :
번역 문장 :
중요 단어 :

규칙 2. 문장 단위로 검토해야 합니다.
    세부규칙 1. 개행 문자 \n를 무시합니다.
    세부규칙 2. 처음부터 가장 처음 나오는 온점('.')이나 느낌표, 물음표 등 마침표('!', '?') 까지를 한 문장으로 보고 끊어가며 첨삭합니다.
    세부규칙 3. 세부규칙 1과 세부규칙2에 해당하지 않을 경우 맺음말(했다, 였다, 이었다, 좋겠다, 있었다 등)로 끊어가며 첨삭합니다.

규칙 3. 요약 문장은 300자 내외로 작성합니다.

규칙 4. 번역 문장은 요약 문장을 번역하고 초등학생이 해석할 수 있는 정도의 난이도로 작성합니다.

규칙 5. 중요 단어는 번역 문장에서 초등학교 고학년 부터 중학생 수준에서 알아두어야 하는 단어를 5개 추출합니다.
"""

In [40]:
# GPT-3.5_turbo 결과 출력
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_prompt_e},
        {"role": "user", "content": user_prompt}, 
    ], temperature = 0.5, top_p= 0.5
)
results = completion.choices[0].message.content
print(results)

요약 문장 : 이 책은 뇌에 대한 정보와 뇌의 기능, 뇌의 구조에 대해 알려주는 책이다. 독자들은 뇌의 신비함과 과학적인 지식을 배우며 즐길 수 있다.

번역 문장 : This book provides information about the brain, its functions, and structure. Readers can learn about the mysteries of the brain and various scientific facts while enjoying the book.

중요 단어 : brain, functions, structure, mysteries, scientific facts


In [21]:
# GPT-4.0 결과 출력
completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt_e},
        {"role": "user", "content": user_prompt}, 
    ], temperature = 0.5, top_p= 0.5
)
results = completion.choices[0].message.content
print(results)

요약 문장 :
이 책은 뇌에 대한 다양한 정보를 제공하며, 뇌의 구조와 기능, 그리고 뇌 세포인 뉴런에 대해 설명하고 있다. 뉴런은 다른 뉴런과 연결되어 있으며, 이 연결을 커넥텀이라고 한다. 사람들의 생각과 감정이 다른 이유는 이 커넥텀이 각각 다르기 때문이다. 또한, 동물들의 뇌에는 피질이 있으며, 이 피질은 동물의 기억력을 결정한다. 사람의 뇌는 주름진 구조를 가지고 있어 장기 기억력이 뛰어나며, 미래를 생각하고 상상할 수 있다.

번역 문장 :
This book provides various information about the brain, explaining the structure and function of the brain, and neurons, the cells of the brain. Neurons are connected to other neurons, and this connection is called a connectome. The reason why people's thoughts and emotions are different is because these connectomes are different for each person. Also, there is a cortex in the brains of animals, and this cortex determines the memory of animals. The human brain has a wrinkled structure, so it has excellent long-term memory, and can think and imagine the future.

중요 단어 :
1. Neurons (뉴런)
2. Connectome (커넥텀)
3. Cortex (피질)
4. Memory (기억력)
5. Future (미래)


In [32]:
import os
import google.generativeai as genai
api_key = ""

genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-pro')

In [26]:
response = model.generate_content(system_prompt_e + user_prompt)
print(response.text)

**요약 문장:**
이 책은 뇌의 구조와 기능에 대한 내용을 알려주는 흥미로운 책입니다. 독자들은 뇌의 신경세포인 뉴런과 기억력에 관여하는 피질에 대해 자세히 알 수 있습니다.

**번역 문장:**
This book is about the brain and how it works. It is short and easy to read, but it is also very interesting and full of information about the brain. You will learn about neurons, which are the cells in your brain, and about the cortex, which is the part of the brain that helps you remember things.

**중요 단어:**
* neuron (뉴런)
* cortex (피질)
* memory (기억)
* connect (연결)
* imagine (상상)


---

2번째 독후감  
(gemini는 몇 번 사용하니 응답이 차단되어 보지 못했습니다....)

---

In [29]:
# GPT-3.5_turbo 결과 출력
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_prompt_e},
        {"role": "user", "content": user_prompt_e}, 
    ], temperature = 0.5, top_p= 0.5
)
results = completion.choices[0].message.content
print(results)

요약 문장 : '와이파이를 먹어버린 펄럭거리는 바지들'이라는 책을 읽고, 인터넷 예절의 중요성을 깨달았다. 이야기 속에서 주인공이 인터넷 예절을 어기며 일어나는 일들을 통해 다른 사람들에게 어떤 피해를 줄 수 있는지 알 수 있었다.

번역 문장 : After reading the book 'The Fluttering Pants That Ate the Wi-Fi', I realized why internet etiquette is so important. Through the story, I learned how the protagonist caused harm to others by not following internet etiquette.

중요 단어 : 
1. internet etiquette - 인터넷 예절
2. harm - 피해
3. hacking - 해킹
4. cyberbullying - 사이버 괴롭힘
5. respect - 존중


In [39]:
# GPT-4결과 출력
completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt_e},
        {"role": "user", "content": user_prompt_e}, 
    ]
)
results = completion.choices[0].message.content
print(results)

요약 문장 : 
이 책은 인터넷 예절의 중요성에 대해 부각한다. 서로 다른 여러 사람이 피해를 입는 상황을 통해 인터넷 활용에 있어서의 예의를 지키는 것이 얼마나 중요한지 강조한다. 책의 내용은 잘못된 인터넷 사용 사례와 그로 인한 되돌릴 수 없는 피해에 대해 읽는 이로 하여금 다시 한 번 고민하게 한다. 또한, 그런 사태를 방지하기 위해서는 우리가 바로 인터넷 예절을 지키게 될 액션을 취해야 함을 말해준다. 빠르게 발전하는 디지털 기술 속에서 더 이상 피해를 주고받지 않기 위해, 인터넷 사용에 있어서 예절을 지키는 것이 얼마나 중요한지 깨닫게 해 주는 책이다.

번역 문장 : 
This book highlights the significance of internet etiquette. It emphasizes how critical it is to be respectful when using the internet, through various situations where different people are harmed. The content of the book provokes a reader to ponder once more about the irreversible harm from wrong uses of the internet. Moreover, it tells us that in order to prevent such situations, we must take immediate action to follow the internet manners. It is a book that makes us realize how important it is to practice proper etiquette when using the internet within a rapidly evolving digital era so as not to inflict or receive any more harm.

중요 단어 : 
1. highlight (강조하다)
2.